In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
from qiskit.primitives import sampler

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler  #Import Sampler
from qiskit_aer import AerSimulator
from qiskit_aer import backends


In [ ]:
#yf = yahoo finance
import yfinance as yf

#(NSE India stocks with .NS suffix)
stocks = ["TCS.NS", "INFY.NS", "HDFCBANK.NS", "RELIANCE.NS", "ITC.NS", "TATAMOTORS.NS", "BAJFINANCE.NS"]

# Download historical stock data for 3 years
df = yf.download(stocks, start="2021-01-01", end="2024-01-01")

#WE ARE USING ONLY THE CLOSE PRICE OF THE STOCKS
df = df['Close']
print(df.head())

[*********************100%***********************]  7 of 7 completed

Ticker      BAJFINANCE.NS  HDFCBANK.NS      INFY.NS      ITC.NS  RELIANCE.NS  \
Date                                                                           
2021-01-01    5204.156738  1368.017578  1148.968262  181.050323   895.606201   
2021-01-04    5141.127441  1359.329712  1174.309448  180.669312   897.115784   
2021-01-05    5045.326172  1369.601440  1179.368774  179.018387   885.962891   
2021-01-06    4957.902832  1363.697632  1168.703613  173.896347   862.598267   
2021-01-07    5007.872559  1359.569702  1150.517944  171.695114   861.201355   

Ticker      TATAMOTORS.NS       TCS.NS  
Date                                    
2021-01-01     185.348572  2683.360840  
2021-01-04     190.118927  2785.260986  
2021-01-05     192.007202  2834.332764  
2021-01-06     194.193604  2796.302979  
2021-01-07     195.535294  2779.167236  


In [10]:
"""we are calculating the daily return and the covariance matrix , the matrix is imp cause it tells us if our portfolio is well balanced or not
if the covariance is high then the stocks are not well balanced and if the covariance is low then the stocks are well balanced"""

returns = df.pct_change()
mean_returns = returns.mean()
cov_mat = returns.cov()


In [15]:
#building the qubo matrix
assets = len(stocks)
Q = np.zeros([assets, assets]) #init the qubo matrix
penalty  = np.mean(abs(mean_returns)) / np.mean(abs(cov_mat.values))
for i in range(assets):
    Q[i,i] = -mean_returns[i]+penalty*cov_mat.iloc[i,i]
    for j in range(i+1, assets):
        Q[i,j] = -cov_mat.iloc[i,j]
        Q[j,i] = -cov_mat.iloc[i,j]

print(Q)   


[[ 2.12437800e-03 -1.19680306e-04 -8.72512907e-05 -7.80136720e-05
  -1.11040710e-04 -1.68183207e-04 -7.11395582e-05]
 [-1.19680306e-04  1.14508629e-03 -6.88003477e-05 -4.47956021e-05
  -7.81294684e-05 -1.13221196e-04 -5.53349492e-05]
 [-8.72512907e-05 -6.88003477e-05  1.28010431e-03 -4.02233253e-05
  -6.99796470e-05 -1.03995006e-04 -1.38913460e-04]
 [-7.80136720e-05 -4.47956021e-05 -4.02233253e-05  1.09301069e-04
  -6.33775257e-05 -1.06829084e-04 -2.94473963e-05]
 [-1.11040710e-04 -7.81294684e-05 -6.99796470e-05 -6.33775257e-05
   1.09533499e-03 -1.34374742e-04 -5.76862894e-05]
 [-1.68183207e-04 -1.13221196e-04 -1.03995006e-04 -1.06829084e-04
  -1.34374742e-04  2.28197068e-03 -9.63110402e-05]
 [-7.11395582e-05 -5.53349492e-05 -1.38913460e-04 -2.94473963e-05
  -5.76862894e-05 -9.63110402e-05  8.08871961e-04]]


C:\Users\Aditya Dwaraki Rao\AppData\Local\Temp\ipykernel_18128\1286306283.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Q[i,i] = -mean_returns[i]+penalty*cov_mat.iloc[i,i]


In [ ]:
#building QAOA to solve qubo matrix
qubo = QuadraticProgram()
for i in range(assets):
    qubo.binary_var(f"x{i}")

#need to convert qubo to optimisation problem
linear_terms =  {f"x{i}": Q[i, i] for i in range(assets)}
quadratic_terms = {(f"x{i}", f"x{j}"): Q[i, j] for i in range(assets) for j in range(i + 1, assets)}
qubo.minimize(linear=linear_terms, quadratic=quadratic_terms)

#lets use QAOA to solve
backend = Aer.get_backend('qasm_simulator')
qaoa = QAOA(optimizer=COBYLA(), reps=1, quantum_instance=backend)
result = qaoa.solve(qubo)


AttributeError: module 'qiskit_aer.backends' has no attribute 'get_backend'